In [1]:
from dotenv import load_dotenv

from processing_functions import get_chunks

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import re

from google import genai

from IPython.display import display, Markdown

import os

import time
import re

In [2]:
load_dotenv()

True

# LLM text preprocessing

In [4]:
system_prompt = """
## CONTEXTO
Você é um especialista em garantias de sistemas prediais. Seu trabalho é analisar informações
do manual do prédio e fornecer, em formato adequado, os dados de prazos de garantia contratual

## FLUXO
O que você irá receber é um texto bruto extraído de um documento. Você deve identificar a tabela de 
prazos de garantia contratual e extrair as informações contidas nessa tabela, prestando atenção nas
instruções abaixo: 

## INSTRUÇÕES
1. Formatação da Tabela no Input
A tabela está em uma estrutura de multiplos cabeçalhos. A primeira coluna contém um cabeçalho único com o título SISTEMAS, ELEMENTOS, COMPONENTES E INSTALAÇÕES.
As colunas 2, 3 e 4 possuem um primeiro cabeçalho com o título PRAZO DE GARANTIA CONTRATUAL, e abaixo desse 
cabeçalho existem os prazos de garantia para cada sistema, elemento, componente ou instalação.

2. Formatação da Tabela no Output
Você dece extrair as informações da tabela da seguinte forma:

SISTEMA X:
    1 ano: Descrição de tudo o que está coberto por esse prazo
    3 anos: Descrição de tudo o que está coberto por esse prazo
    5 anos: Descrição de tudo o que está coberto por esse prazo
Faça isso para todos os sistemas, elementos, componentes e instalações que existirem na tabela.

3. Execute a tarefa passo a passo:
- Extraia o texto bruto da tabela
- Identifique os sistemas, elementos, componentes e instalações
- Extraia os prazos de garantia contratual e suas descrições
- Formate a saída de acordo com as instruções acima
- Certifique-se que você extraiu a tabela completa

Texto com as tabelas:


"""

In [5]:
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

In [6]:
def load_txt_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()


tabela_8_text = load_txt_file("tabela_8.txt")

In [7]:
response = client.models.generate_content(
    model="gemini-2.5-pro", contents=[system_prompt, tabela_8_text]
)

In [8]:
display(Markdown(response.text))

Com certeza! Como especialista em garantias de sistemas prediais, analisarei o texto fornecido e extrairei as informações conforme solicitado.

Aqui está a análise dos prazos de garantia contratual, formatada de acordo com as suas instruções:

***

**FUNDAÇÕES:**
*   **5 anos:** Recalque, afundamento, trincas estruturais, umidade ascendente, reação álcali-agregado.

**ESTRUTURA DE CONCRETO ARMADO E PROTENDIDO:**
*   **5 anos:** Integridade do concreto (trincas/fissuras/rachaduras brocas/vazios/deformações/desagregação de componentes/esmagamento/carbonatação/formação de estalactites e estalagmites); Integridade da armadura (despassivação da armadura, processo de corrosão, recobrimento da armadura insuficiente); Ausência ou insuficiência de sistemas de proteção (revestimentos hidrofugantes e tratamentos superficiais); Formação de estalactites e estalagmites devido a exposição a ambientes agressivos; Ausência ou insuficiência de juntas de dilatação/movimentação e selantes; Falhas de execução (defeitos na retirada de formas e escoramento, resistência (FCK) abaixo do previsto); Flechas excessivas por falhas de projeto ou executivas; Defeitos na junção entre componentes de diferentes materiais (problemas de estanqueidade/trincas).

**ESTRUTURAS METÁLICAS (ESTRUTURA INTERNA DO PÓRTICO):**
*   **5 anos:** Integridade da estrutura metálica (deformações); Flechas excessivas por falhas de projeto ou executivas; Fraturas e propagação de fraturas; Falhas por má concepção de projeto quanto à qualidade e desempenho do aço; Falta de concordância em emendas; Defeitos na junção entre componentes de diferentes materiais (problemas de estanqueidade/trincas).

**ALVENARIA DE VEDAÇÃO EM BLOCOS CERÂMICOS OU DE CONCRETO:**
*   **5 anos:** Trincas e rachaduras, problemas de alinhamento e paginação por movimentação estrutural e defeitos de encunhamento; Infiltrações por movimentação estrutural; Ausência/subdimensionamento de contra vergas, amarração da alvenaria.

**PAREDE EM DRYWALL:**
*   **2 anos:** Trincas e fissuras.

**IMPERMEABILIZAÇÃO:**
*   **5 anos:** Estanqueidade.

**DISPOSITIVOS DE PROTEÇÃO TÉRMICA PARA RETRAÇÃO E DILATAÇÃO (JUNTAS DE DILATAÇÃO/MOVIMENTAÇÃO):**
*   **1 ano:** Aderência.
*   **5 anos:** Integridade estrutural do conjunto.

**ESQUADRIAS DE ALUMÍNIO:**
*   **1 ano:** Fixação, desencaixe ou deformação; Mau funcionamento.
*   **2 anos:** Problemas com a instalação, vedação, ou desempenho do material (borrachas, escovas, articulações, fechos e roldanas).
*   **5 anos:** Integridade estrutural do conjunto (perfil) decorrente das falhas de fabricação, tais como rupturas, deformação por flexão.

**ESQUADRIAS METÁLICAS (FERRO E AÇO):**
*   **1 ano:** Fixação; Oxidação impacta nas condições de integridade estrutural do conjunto e o uso de proteção/pintura adequada garante os 5 anos, conforme informação do fabricante.
*   **2 anos:** Vedação/estanqueidade/instalação (patologias e não características).
*   **5 anos:** Integridade estrutural do conjunto decorrente das falhas de fabricação, tais como peças soltas, elementos empenados, falta de esquadro/oxidação.

**ESQUADRIAS DE MADEIRA:**
*   **1 ano:** Degradação por agentes biológicos (cupins/insetos); Empenamento com ou sem ocorrência de fendas/trincas; Baixa qualidade quanto ao desempenho a esforços mecânicos; Instalação inadequada, inclusive de elementos de fixação; Descolamento de fitas, guarnições, escovas e elementos de fixação.

**FERRAGENS DAS ESQUADRIAS (MAÇANETAS, FECHOS E ARTICULAÇÕES):**
*   **1 ano:** Baixa qualidade quanto ao desempenho a esforços mecânicos; Oxidação e processos corrosivos; Instalação inadequada.

**PERGOLADO DE MADEIRA:**
*   **1 ano:** Degradação por agentes biológicos (cupins/insetos); Empenamento com ou sem ocorrência de fendas/trincas; Baixa qualidade quanto ao desempenho a esforços mecânicos; Instalação inadequada, inclusive de elementos de fixação; Descolamento de fitas, guarnições, escovas e elementos de fixação.

**GUARDA CORPO E COMPONENTES DE ANCORAGEM:**
*   **1 ano:** Oxidação e processos corrosivos que não afetem a perda de seção da peça e sua estabilidade.
*   **5 anos:** Ruptura ou perda de estabilidade.

**CORRIMÃO METÁLICO:**
*   **1 ano:** Oxidação e processos corrosivos que não afetem a perda de seção da peça e sua estabilidade.
*   **3 anos:** Ruptura ou perda de estabilidade; Instalação inadequada dos elementos de fixação e falha de fabricação.

**ESTRUTURA DE COBERTURA EM ELEMENTOS METÁLICOS:**
*   **5 anos:** Deformações por uso de material de baixa qualidade quanto ao desempenho a esforços mecânicos; Subdimensionamento dos elementos estruturantes; Cálculo de declividade (angulação); Estanqueidade; Execução inadequada dos elementos de fixação e soldas; Processos corrosivos.

**TELHAS (CERÂMICA, FIBROCIMENTO, METÁLICA, ALUMÍNIO, CONCRETO, PVC):**
*   **1 ano:** Deformações, empenamentos e fissuras, além dos limites da norma técnica e recomendações do fabricante.
*   **3 anos:** Dessolidarização ou ruptura.
*   **5 anos:** Integridade do elemento e seus componentes.

**SISTEMA DE DRENAGEM PLUVIAL (CALHAS, RUFOS, TUBOS E PEÇAS ACESSÓRIAS):**
*   **1 ano:** Falha de fixação e perda de estanqueidade.
*   **5 anos:** Integridade da estrutura de calhas, rufos, tubos e peças acessórias.

**PISO CIMENTADO, PISO ACABADO EM CONCRETO, CONTRAPISO:**
*   **2 anos:** Superfície irregular; Falhas no caimento ou nivelamento.
*   **5 anos:** Ausência de juntas de dilatação; Desgaste excessivo; Fissuras; Destacamentos.

**PISO CERÂMICO:**
*   **1 ano:** Ausência de declividade do piso em sentido aos elementos de escoamento (ralos/grelhas e caixas sifonadas).
*   **2 anos:** Baixa qualidade quanto ao desempenho a esforços mecânicos; Desplacamento, peças gretadas ou desgaste excessivo.
*   **3 anos:** Estanqueidade do sistema de pisos nas áreas molháveis.

**PISO LAMINADO:**
*   **1 ano:** Trincados; Baixa qualidade quanto ao desempenho a esforços mecânicos (desgaste precoce); Instalação inadequada dos elementos de fixação; Irregularidade da superfície por instalação inadequada ou desnivelamento do contrapiso.

**REJUNTAMENTO:**
*   **1 ano:** Desprendimento (má aderência).

**REVESTIMENTO EM PEDRAS (MÁRMORE, GRANITO, ETC):**
*   **1 ano:** Execução inadequada do assentamento (formação de vazios na face posterior da peça); Desplacamento, peças gretadas ou desgaste excessivo.
*   **2 anos:** Baixa qualidade quanto ao desempenho a esforços mecânicos.
*   **5 anos:** Má aderência do revestimento e dos componentes do sistema (fixação dos elementos); Eflorescência causada por uso de argamassa inadequada (que provoca a precipitação de sais na superfície do revestimento).

**REVESTIMENTOS EM ARGAMASSA, GESSO OU ARGAMASSAS DECORATIVAS (TEXTURAS):**
*   **1 ano:** Perda prematura de coloração pela baixa qualidade do pigmento do material utilizado.
*   **2 anos:** Empolamento, descascamento, esfarelamento, deterioração.
*   **3 anos:** Estanqueidade de fachadas.
*   **5 anos:** Má aderência do revestimento e dos componentes do sistema.

**PINTURA/ VERNIZ/ SELANTES:**
*   **1 ano:** Infiltração decorrente da ação da umidade.
*   **2 anos:** Empolamento, esfarelamento, descascamento, alteração de cor ou deterioração de acabamento; Ruptura da superfície por má execução do substrato (emboço/reboco).
*   **5 anos:** Ruptura da superfície por movimentação estrutural/térmica.

**FORRO DE GESSO:**
*   **1 ano:** Manchas, mofo e bolor por ação da umidade (vazamentos); Instalação inadequada dos elementos de fixação; Deformação da estrutura de sustentação do forro; Manchas de oxidação da estrutura de sustentação do forro; Ruptura da superfície decorrente da má execução do substrato (emboço/reboco).

**VIDRO:**
*   **1 ano:** Defeito de instalação, fixação.

**SISTEMA DE AR-CONDICIONADO:**
*   **1 ano:** Falha nos produtos/materiais.
*   **3 anos:** Falhas de instalação da infraestrutura.

**INSTALAÇÕES ELÉTRICAS - (TOMADAS, INTERRUPTORES, DISJUNTORES, FIOS, CABOS, ELETRODUTOS, CAIXAS E QUADROS):**
*   **3 anos:** Baixa qualidade do material; Incompatibilidade entre a bitola do condutor e a potência nominal do equipamento utilizado e subdimensionamento do circuito para atender a demanda prevista; Ausência de dispositivo de segurança contra choques elétricos (DR); Deficiência nos procedimentos de isolamento e emendas e falhas na instalação dos componentes elétricos; Ausência ou precariedade na realização de aterramento; Obstrução total ou parcial de eletrodutos e caixas de passagem; A execução não corresponde às definições do projeto quanto à cor de fiação, dimensionamento das redes e balanceamentos das fases; Falta de identificação dos circuitos nos quadros de distribuição; Instalação fios/cabos/tubulações/disjuntor.

**SISTEMA DE ILUMINAÇÃO:**
*   **1 ano:** Baixa qualidade de material ou especificação inadequada; Desempenho inferior ao especificado; Problemas de instalação (reatores/soquetes/isolamento/ocorrência de pontos quentes).

**INSTALAÇÕES HIDRÁULICAS:**
*   **2 anos:** Engates flexíveis, sifão, válvulas, ralos e seus acabamentos; Desempenho inferior ao especificado (falha no material).
*   **3 anos:** Baixa qualidade do material; Incompatibilidade entre diâmetro da tubulação e vazão necessária para atender a demanda prevista; Deficiência nos procedimentos de assentamento das tubulações, sobretudo no uso de adesivos em peças de conexões soldáveis e fita veda-rosca nas peças rosqueáveis; Falta de previsão de tubulações para extravasamento e limpeza/manutenção; A execução não corresponde às definições do projeto.
*   **5 anos:** Erro no projeto de impermeabilização/dimensionamento de reservatório/cisterna; Integridade e estanqueidade de colunas de água fria e colunas de água quente; Vazamentos por ruptura na tubulação ou desencaixe de conexões; Perda de carga decorrente de erro de cálculo na concepção do projeto ou execução em desacordo com as especificações previstas.

**INSTALAÇÕES DE ESGOTO:**
*   **3 anos:** Vazamentos, extravasamento; Subdimensionamento do diâmetro da tubulação para atender a vazão de escoamento prevista gerando refluxo/retorno; Vazamentos por ruptura na tubulação, desencaixe de conexões e falha de vedação; Ausência ou insuficiência de dispositivos de ventilação (caixa sifonada/tubos/sifão); Subdimensionamento ou ausência de caixas de gordura, ralos, grelhas; A execução não corresponde às definições do projeto; Obstruções na tubulação decorrente de resíduos da obra (ex. torrões de argamassa/fragmentos de material de construção).
*   **5 anos:** Integridade e estanqueidade de tubos de queda; Deficiência nos procedimentos de assentamento das tubulações; Falha no cálculo de declividade (vício de projeto); Erro na especificação (curvas inapropriadas) e instalação de conexões (descolamento).

**INSTALAÇÕES DE DRENAGEM PLUVIAL:**
*   **3 anos:** Os vícios são verificados imediatamente, quando constatados problemas de funcionamento, mas a maioria dos casos são ocultos, evidenciados por vazamentos e extravasamento; Falha no cálculo de declividade (vício de projeto); Erro na especificação (curvas inapropriadas) e instalação de conexões (descolamento); Vazamentos por ruptura na tubulação, desencaixe de conexões e falhas de vedação; Subdimensionamento ou ausência de caixas de inspeção, ralos, grelhas; A execução não corresponde às definições do projeto; Obstruções na tubulação decorrente de resíduos da obra (ex. torrões de argamassa/fragmentos de material de construção).
*   **5 anos:** Deficiência nos procedimentos de assentamento das tubulações.

**LOUÇAS (BACIAS SANITÁRIAS/ PIAS/ TANQUES):**
*   **1 ano:** Desempenho inferior ao especificado (defeito no equipamento).
*   **3 anos:** Uso de material de baixa qualidade (quebra e desprendimento das peças); Falhas de vedação; Problemas na instalação.

**METAIS (TORNEIRAS/ VÁLVULAS DE DESCARGA/ REGISTROS):**
*   **1 ano:** Uso de material de baixa qualidade (quebra e desprendimentos das peças); Desempenho inferior ao especificado (defeito no equipamento).
*   **2 anos:** Falhas de vedação.
*   **3 anos:** Problemas na instalação; Processo de oxidação/de acordo com orientações do fabricante.

**COMPONENTES E ACESSÓRIOS PARA ACESSIBILIDADE (BARRAS DE APOIO, PISOS TÁTEIS E DIRECIONAIS, ASSENTOS ESPECIAIS, SINALIZAÇÕES, ALARMES VISUAIS E SONOROS):**
*   **1 ano:** Falhas de produto ou instalação.

**MOTOBOMBAS - SISTEMA DE RECALQUE (REDE DE ABASTECIMENTO DE ÁGUA/ ESTAÇÃO ELEVATÓRIA DE ESGOTO):**
*   **1 ano:** Uso de equipamentos de baixa qualidade (desgaste prematuro de componentes); Desempenho inferior ao especificado (defeito no equipamento); Problemas na instalação (ex. posicionamento de sensores de nível, fixação, nivelamento, alimentação elétrica, condutores com bitolas subdimensionadas); Erros no dimensionamento do equipamento (potência inadequada para a demanda).

**SISTEMA DE AQUECIMENTO CENTRAL DE ÁGUA (ELÉTRICO/ CALDEIRA/GÁS/ SOLAR):**
*   **1 ano:** Uso de equipamentos de baixa qualidade (desgaste prematuro de componentes); Desempenho inferior ao especificado (defeito no equipamento).
*   **3 anos:** Problemas na instalação (ex. posicionamento de sensores de nível, fixação, nivelamento, alimentação elétrica, condutores com bitolas subdimensionadas).
*   **5 anos:** Erros no dimensionamento do equipamento (potência inadequada para a demanda); Em relação ao SAS, erro no posicionamento das placas de captação da energia em relação à incidência do sol e falha na previsão de sistema de aquecimento contingência; Falha no isolamento térmico dos reservatórios (boiler); Condutores e conexões não apropriados para água quente causando perda de calor.

**SISTEMA DE GÁS:**
*   **1 ano:** Compartimento de armazenamentos de gás com falhas quanto à segurança e ausência de medidores individuais; Uso de material de baixa qualidade.
*   **3 anos:** Problemas na instalação dos ramais (ex. instalados em locais sem segurança); Execução em desacordo com projeto/normas técnicas; Estanqueidade e vazamento (falha por vedação e defeito em conectores de extremidade).
*   **5 anos:** Integridade e estanqueidade de colunas de gás.

**INSTALAÇÃO TELEFÔNICA:**
*   **1 ano:** Uso de material de baixa qualidade; Obstrução da tubulação de passagem; Falha do dimensionamento da tubulação de passagem; Tubulação e cabeamentos externos.

**INSTALAÇÕES LÓGICAS (CABEAMENTO ESTRUTURADO):**
*   **1 ano:** Uso de material de baixa qualidade; Obstrução da tubulação de passagem; Falha do dimensionamento da tubulação de passagem; Tubulação e cabeamentos externos; Falhas de instalação.

**INSTALAÇÃO INTERFONIA:**
*   **1 ano:** Obstrução da tubulação de passagem de fiação; Falhas de instalação; Funcionamento dos equipamentos internos e portaria; Acabamento/aparência.

**INSTALAÇÃO DE COMBATE A INCÊNDIO:**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação.
*   **5 anos:** Subdimensionamento da reserva técnica de incêndio (reservatórios).

**PORTA CORTA-FOGO:**
*   **1 ano:** Integridade dos componentes (batentes, molas, dobradiças); Molas, dobradiças, barras antipânico ou maçaneta: Mau funcionamento, fixação e corrosão.
*   **3 anos:** Folha da porta e marcos (batentes): deformação, ruptura; dessolidarização.

**ILUMINAÇÃO DE EMERGÊNCIA:**
*   **1 ano:** Instalação / equipamentos.

**SISTEMA DE PROTEÇÃO PARA DESCARGAS ATMOSFÉRICAS - SPDA:**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação.
*   **5 anos:** Falhas de dimensionamento no sistema.

**ILUMINAÇÃO AUTOMÁTICA (SENSOR DE PRESENÇA):**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação.

**ELEVADORES:**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação; Falhas de dimensionamento no sistema (capacidade de atendimento).

**PISCINA:**
*   **1 ano:** Problemas de qualidade do material (revestimentos e tubulação de ligação); Problemas decorrentes do desempenho do equipamento (defeitos prematuros - bombas/sensores/quadros de comando/filtros); Desgaste e dessolidarização rejuntamento, sistemas de juntas e seus componentes.
*   **3 anos:** Falhas de instalação dos componentes hidráulicos; Falhas e dessolidarização do revestimento.
*   **5 anos:** Estanqueidade.

**ANTENA COLETIVA:**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação.

**SISTEMA DE AUTOMAÇÃO DE PORTÕES:**
*   **1 ano:** Problemas decorrentes do desempenho do equipamento (defeitos prematuros); Falhas de instalação.

**SINALIZAÇÃO PARA ACESSIBILIDADE/ INCÊNDIO E IDENTIFICAÇÕES EM GERAL:**
*   Este item não possui prazos de garantia especificados nas colunas de 1, 2, 3 ou 5 anos no texto fornecido.

**QUADRAS POLIESPORTIVAS:**
*   Este item não possui prazos de garantia especificados nas colunas de 1, 2, 3 ou 5 anos no texto fornecido.

**INFRAESTRUTURA EXTERNA DE PROTEÇÃO - CONTENÇÕES E ARRIMOS:**
*   **5 anos:** Deslizamentos, rupturas e tombamento de maciços terrosos; Não atendimento das normas relacionadas a distanciamento entre taludes e edificações; Não atendimento das inclinações normatizadas para cortes e aterros em taludes; Erosão decorrente da ausência ou insuficiência de cobertura vegetal; Opção por soluções de menor custo, porém inadequadas para evitar os processos erosivos no local; Erros nos cálculos relativos a área de contribuição, intensidade pluviométrica, períodos de retorno e coeficiente de deflúvio (escoamento).

**PAVIMENTAÇÃO:**
*   **5 anos:** Uso de material de baixa qualidade ou inadequado; Falhas executivas (falhas no processo de compactação/imprimação/não aplicação de emulsão asfáltica para rejuntamento); Desgaste prematuro do material; Recalques; Desagregação superficial do pavimento; Ausência ou insuficiência de declividade transversal e/ou longitudinal em projetos/execução.

**PISOS DE AMBIENTES EXTERNOS:**
*   **3 anos:** Pisos cobertos e descobertos de estacionamentos/garagens externos ao edifício: Desgaste; dessolidarização; ruptura; deterioração por umidade.

**PAVIMENTAÇÃO EXTERNA À EDIFICAÇÃO:**
*   **1 ano:** Pavimentos de acesso de automóveis à edificação: Desgaste; dessolidarização.
*   **2 anos:** Pavimentos de acesso de veículos de carga e descarga: Desgaste; dessolidarização.
*   **3 anos:** Pavimentos de acesso de pedestres à edificação: Desgaste; dessolidarização.

**INFRA-ESTRUTURA EXTERNA DE DRENAGEM PLUVIAL - (SARJETAS/ DRENOS/ BOCAS DE LOBO APROPRIADAS/ POÇOS DE VISITAS/ TUBOS DE ESCOAMENTO):**
*   **5 anos:** Subdimensionamento do sistema de drenagem decorrente de erros nos cálculos relativos a área de contribuição, intensidade pluviométrica, períodos de retorno e coeficiente de deflúvio (escoamento); Destinação indevida das águas captadas, resultando em alagamentos; Alagamentos e extravasamentos devido à sistema mal dimensionado; Erosão decorrente da ausência ou insuficiência do sistema de drenagem; Deterioração da pavimentação decorrente de falhas no sistema de drenagem; Degradação das edificações por sistema de drenagem externa ineficiente.

**INFRA-ESTRUTURA EXTERNA - DIVISAS (MUROS/ CERCAS/ ALAMBRADOS):**
*   **1 ano:** Cercas, alambrados ou conforme indicação do fabricante; Deterioração por umidade.
*   **3 anos:** Fissuração.
*   **5 anos:** Material de baixa qualidade; Falha nos procedimentos executivos que causam ruptura ou tombamento; Não atendimento aos limites dimensionais (alturas limites) definidos na legislação municipal (código de obras local); Instabilidade estrutural.

**ESCADAS:**
*   **5 anos:** Trincas e afundamentos; Vibração excessiva; Problemas de fixação e desplacamentos de elementos estruturais.

In [10]:
print(response.text)

Com certeza! Como especialista em garantias de sistemas prediais, analisarei o texto fornecido e extrairei as informações conforme solicitado.

Aqui está a análise dos prazos de garantia contratual, formatada de acordo com as suas instruções:

***

**FUNDAÇÕES:**
*   **5 anos:** Recalque, afundamento, trincas estruturais, umidade ascendente, reação álcali-agregado.

**ESTRUTURA DE CONCRETO ARMADO E PROTENDIDO:**
*   **5 anos:** Integridade do concreto (trincas/fissuras/rachaduras brocas/vazios/deformações/desagregação de componentes/esmagamento/carbonatação/formação de estalactites e estalagmites); Integridade da armadura (despassivação da armadura, processo de corrosão, recobrimento da armadura insuficiente); Ausência ou insuficiência de sistemas de proteção (revestimentos hidrofugantes e tratamentos superficiais); Formação de estalactites e estalagmites devido a exposição a ambientes agressivos; Ausência ou insuficiência de juntas de dilatação/movimentação e selantes; Falhas de execu


**Olá! Esse é um teste para ver como mds se comportam**
***